In [1]:
import numpy as np  # whole numpy lib is available, prepend 'np.'
from numpy import (sin, cos, tan, log, log10, pi, average,
                   sqrt, std, deg2rad, rad2deg, linspace, asarray)
from numpy.random import random, randint, normal, shuffle, choice as randchoice
import pandas as pd

import os  # handy system and path functions
import sys  # to get file system encoding

from psychopy import  data


In [5]:
btext = 'Block break \n\n'
correct_rate= 80
num_block = 2

# concatenate correct_rate and num_block to the text
btext = btext + 'Correct rate: ' + str(correct_rate) + '%\t' + 'Number of blocks: ' + str(num_block)
print(btext)

Block break 

Correct rate: 80%	Number of blocks: 2


In [48]:
trials = data.TrialHandler(nReps=1.0, method='sequential', 
     originPath=-1,
    trialList=data.importConditions("data/seq_str.csv"),
    seed=None, name='trials')
thisTrial = trials.trialList[0]  # so we can initialise stimuli with some values
for paramName in thisTrial:
    exec('{} = thisTrial[paramName]'.format(paramName))


In [14]:
expInfo = {'distractor_prevalence':1}

In [15]:
n_trials = 600
# Create item locations, each row contains random shuffled numbers from 0 to 5 (first Target, second distractor)
itempos = np.array([randchoice(6, 6, replace=False) for i in range(n_trials)])
# create distractor shapes with 5 columns, each row contains random samples from [3,5,6,0,3,5,6,0] 
# 3,5,6,0(triangle, pentagon, hexagon, circle)
item_shapes = np.array([randchoice([3,5,6,0,3,5,6,0], 5, replace=False) for i in range(n_trials)])
# add a column of 4s to the first of the matrix
item_shapes = np.insert(item_shapes, 0, 4, axis=1)


# create target color, and target orietnation [0,1]
# initialize a matrix with half 0s and half 1s
item_specs = np.zeros((n_trials, 2))
# specify the second half of the matrix to be 1s
item_specs[n_trials//2:, :] = 1
# random shuffle rows, independent of each column
for i in range(2):
    np.random.shuffle(item_specs[:, i])

# singleton prevalence
nsingleton = np.int16(expInfo['distractor_prevalence']*n_trials)
singleton_trials = np.concatenate((np.ones((nsingleton, 1)), 
    np.zeros((n_trials-nsingleton, 1))), axis=0)
np.random.shuffle(singleton_trials)
# add cue validity of 80 percent by generating 80% 1s and 20% 0s, as one column
cue_valid_trials = np.int16(0.9*n_trials)
item_valid = np.concatenate((np.ones((cue_valid_trials, 1)), np.zeros((n_trials-cue_valid_trials, 1))), axis=0)
# random shuffle rows
np.random.shuffle(item_valid)

# combine itempos, item_shapes, and item_specs into trials
sequences = np.concatenate((itempos, item_shapes, singleton_trials, item_specs, item_valid), axis=1)
# # convert trials to pandas dataframe
sequences = pd.DataFrame(sequences, columns=['pos1', 'pos2', 'pos3', 'pos4', 'pos5', 'pos6', 
                                       'shape1', 'shape2', 'shape3', 'shape4', 'shape5', 'shape6',
                                       'singleton', 'target_color', 'target_orientation', 'cue_validity'])
# change all columns to int
sequences = sequences.astype(int)

#save trials to csv file in subfolder sequences
#trials.to_csv('sequences/trials.csv', index=False)
sequences.head(20)




,pos1,pos2,pos3,pos4,pos5,pos6,shape1,shape2,shape3,shape4,shape5,shape6,singleton,target_color,target_orientation,cue_validity
0,4,1,2,0,5,3,4,5,3,5,6,0,1,0,1,1
1,2,3,4,0,5,1,4,3,6,0,5,6,1,1,0,1
2,3,2,1,0,4,5,4,3,6,3,5,0,1,1,0,1
3,4,0,5,2,3,1,4,3,6,5,5,0,1,1,0,0
4,4,0,3,5,2,1,4,5,3,0,0,6,1,0,0,1
5,3,4,1,5,2,0,4,6,6,3,3,5,1,0,1,1
6,0,2,5,4,3,1,4,0,3,6,6,0,1,1,0,1
7,2,0,3,4,1,5,4,6,0,6,3,5,1,0,1,1
8,5,4,3,1,2,0,4,3,5,3,6,0,1,0,1,0
9,5,0,2,3,1,4,4,0,0,6,3,5,1,1,1,1


In [16]:
a = '50'
# convert a to float
float(a)

50.0

## Trials for diamond shape search

This is a simplified version for low-functioning (IQ < 70) ASD children. The task is to search for a diamond shape target among distractors, whether it is located on the left or the right side of the screen, by pressing the left and right arrow key, respectively.

In [11]:
# the experiment need to be finished within 10 minutes, so the number of trials set to 120, with each block of 30 trials
n_trials = 120
block_trials = 30
num_block = n_trials//block_trials

# there are 6 possible locations for the target and distractors, [0, 1,3,4] for target, [2,5] for distractors
# create an array of 120 rows with possible target locations
target_pos = np.array([randchoice([0,1,3,4], 1, replace=False) for i in range(n_trials)])
# create an array of 120 rows with possible distractor locations
distractor_pos = np.array([randchoice([2,5], 1, replace=False) for i in range(n_trials)])
# create any array of 120 rows, 5 columns with possible shapes for distractors, 3, 5, 6, 0 (triangle, pentagon, hexagon, circle)
distractor_shapes = np.array([randchoice([3,5,6,0], 5, replace=True) for i in range(n_trials)])
# create an array of 120 row with possible distractor presence (0,1)
singleton = np.array([randchoice([0,1], 1, replace=True) for i in range(n_trials)])
# create an array of 120 rows with possible target colors (0,1)
target_color = np.array([randchoice([0,1], 1, replace=True) for i in range(n_trials)])
# create correct answer 2D array of 120 rows, one column based on target_pos: > 2 is 0, < 2 is 1 (left, right) with one column
correct_answer = np.array([0 if i > 2 else 1 for i in target_pos])
# convert correct_answer to 2D array
correct_answer = correct_answer.reshape(n_trials, 1)

# concatenate all arrays into one array
sequences = np.concatenate((target_pos, distractor_pos, distractor_shapes, singleton, target_color, correct_answer), axis=1)
# convert sequences to pandas dataframe
sequences = pd.DataFrame(sequences, columns=['target_pos', 'distractor_pos', 'distractor_shape1', 'distractor_shape2', 
                                             'distractor_shape3', 'distractor_shape4', 'distractor_shape5', 
                                             'singleton', 'target_color', 'correct_answer'])
# change all columns to int
sequences = sequences.astype(int)
# save sequences to csv file in subfolder data
#sequences.to_csv('data/seq_str.csv', index=False)
sequences.head(20)



,target_pos,distractor_pos,distractor_shape1,distractor_shape2,distractor_shape3,distractor_shape4,distractor_shape5,singleton,target_color,correct_answer
0,4,2,3,3,0,5,5,0,0,0
1,1,2,0,5,5,6,3,1,1,1
2,3,2,3,0,0,3,3,0,0,0
3,3,5,3,5,5,3,0,1,1,0
4,1,5,0,3,0,0,6,1,1,1
5,3,2,5,6,0,0,5,1,0,0
6,3,2,0,3,6,0,0,0,0,0
7,4,2,0,0,5,0,3,1,1,0
8,0,2,3,0,3,0,0,0,1,1
9,0,2,3,5,3,6,5,0,0,1


In [ ]:
# convert a integer to string, concatename with 'pos'
str(1) + 'pos'
